## [Asynchronous I/O, event loop, and concurrency tools](https://pymotw.com/3/asyncio/index.html)

## Asynchronous Concurrency Concepts
**other concurrency model**: rely on threading or process management of the language runtime or operating system to change context

**asyncio**: explicitly handle context changes using eventloop and coroutine

**eventloop** - a place to register code to be run in application.

**coroutines** - as a mechanism for yielding control back to the event loop, these are special functions that give up control to their caller without losing their state (implented with generator in version older than Python 3.5)

**Future** - a data sturucture representing the result of work that has not been completed yet.

**Task** - a subclass of Future that knows how to wrap and manage the execution for a coroutine, can be scheduled with event loop.

## Cooperative Multitasking with Coroutines
Coroutines 
- a language construct designed for concurrency.
- creates a coroutine object when they called.
- then caller can run it using coroutine's `send()` method.
- while it is paused, their stats is manintaied
- when awakened, coroutines resume where it left off the next tinme.

### Starting a Coroutine

In [1]:
# asyncio_coroutine.py
import asyncio


async def coroutine():
    print('in coroutine')


event_loop = asyncio.get_event_loop()
try:
    print('starting coroutine')
    coro = coroutine()
    event_loop.run_until_complete(coro)
finally:
    print('closing event loop')
    # event_loop.close() - skipped in jupyter

starting coroutine
in coroutine
closing event loop


### Returning Values from Coroutines

In [2]:
# asyncio_coroutine_return.py
import asyncio


async def coroutine():
    print('in coroutine')
    return 'result'


event_loop = asyncio.get_event_loop()
try:
    return_value = event_loop.run_until_complete(
        coroutine()
    )
    print('it returned: {!r}'.format(return_value))
finally:
    pass
    # event_loop.close() - skipped in jupyter

in coroutine
it returned: 'result'


### Chaning Coroutines
decompising a task into resuable parts.

In [3]:
# asyncio_coroutine_chain.py
import asyncio


async def outer():
    print('in outer')
    print('waiting for result1')
    result1 = await phase1()
    print('waiting for result2')
    result2 = await phase2(result1)
    return (result1, result2)

async def phase1():
    print('in phase1')
    return 'result1'

async def phase2(arg):
    print('in phase2')
    return 'result2 derived from {}'.format(arg)

event_loop = asyncio.get_event_loop()
try:
    return_value = event_loop.run_until_complete(outer())
    print('return value: {!r}'.format(return_value))
finally:
    pass
    # event_loop.close() - skipped in jupyter

in outer
waiting for result1
in phase1
waiting for result2
in phase2
return value: ('result1', 'result2 derived from result1')


### Generators Instead of Coroutines
Earlier versions of Python 3 can use generator functions with the `asyncio.coroutine()` decorator and `yield from`

In [4]:
# asyncio_generator.py
import asyncio


@asyncio.coroutine
def outer():
    print('in outer')
    print('waiting for result1')
    result1 = yield from phase1()
    print('waiting for result2')
    result2 = yield from phase2(result1)
    return (result1, result2)


@asyncio.coroutine
def phase1():
    print('in phase1')
    return 'result1'


@asyncio.coroutine
def phase2(arg):
    print('in phase2')
    return 'result2 derived from {}'.format(arg)


event_loop = asyncio.get_event_loop()
try:
    return_value = event_loop.run_until_complete(outer())
    print('return value: {!r}'.format(return_value))
finally:
    pass
    # event_loop.close()

in outer
waiting for result1
in phase1
waiting for result2
in phase2
return value: ('result1', 'result2 derived from result1')


## Scheduling Calls to Regular Functions
schedule calls to regular functions based on the timer value in the loop.

In [5]:
# asyncio_call_soon.py
import asyncio
import functools


def callback(arg, *, kwarg='default'):
    print('callback invoked with {} and {}'.format(arg, kwarg))
    
async def main(loop):
    print('registering callbaks')
    loop.call_soon(callback, 1)
    wrapped = functools.partial(callback, kwarg='not default')
    loop.call_soon(wrapped, 2)
    
    await asyncio.sleep(0.1)
    
event_loop = asyncio.get_event_loop()
try:
    print('entering event loop')
    event_loop.run_until_complete(main(event_loop))
finally:
    print('closing event loop')
    # event_loop.close()

entering event loop
registering callbaks
callback invoked with 1 and default
callback invoked with 2 and not default
closing event loop


### Scheduling a Callback with a Delay

In [6]:
# asyncio_call_later.py

import asyncio

def callback(n):
    print('callback {} invoked'.format(n))
    
async def main(loop):
    print('registering callbacks')
    loop.call_later(0.2, callback, 1)
    loop.call_later(0.1, callback, 2)
    loop.call_soon(callback, 3)
    
    await asyncio.sleep(0.4)
    
event_loop = asyncio.get_event_loop()
try:
    print('entering event loop')
    event_loop.run_until_complete(main(event_loop))
finally:
    print('closing event loop')
    # event_loop.close()

entering event loop
registering callbacks
callback 3 invoked
callback 2 invoked
callback 1 invoked
closing event loop


### Scheduling a Callback for a Specific Time
Use `time()` method of event loop to start the internal state of clock; event loop uses a monotonic clock, not a wall-clock time, to ensure that value of "now" never regresses.

In [7]:
# asyncio_call_at.py

import asyncio
import time


def callback(n, loop):
    print('callback {} invoked at {}'.format(n, loop.time()))
    
    
async def main(loop):
    now = loop.time()
    print('clock time: {}'.format(time.time()))
    print('loop  time: {}'.format(now))
    
    print('registering callbacks')
    loop.call_at(now + 0.2, callback, 1, loop)
    loop.call_at(now + 0.1, callback, 2, loop)
    loop.call_soon(callback, 3, loop)
    
    await asyncio.sleep(0)
    

event_loop = asyncio.get_event_loop()
try:
    print('entering event loop')
    event_loop.run_until_complete(main(event_loop))
finally:
    print('closing event loop')
    # event_loop.close()

entering event loop
clock time: 1487861185.7658634
loop  time: 6699.540989305
registering callbacks
callback 3 invoked at 6699.541720254
closing event loop


## Producing Results Asynchronously
Future - the result of work that has not been completed yet.

### Waiting for a Future


In [8]:
# asyncio_future_event_loop.py
import asyncio


def mark_done(future, result):
    print('setting future result to {!r}'.format(result))
    future.set_result(result)

    
event_loop = asyncio.get_event_loop()
try:
    all_done = asyncio.Future()
    
    print('scheduling mark done')
    event_loop.call_soon(mark_done, all_done, 'the result')
    
    print('entering event loop')
    result = event_loop.run_until_complete(all_done)
    print('returning result: {!r}'.format(result))
finally:
    print('closing event loop')
    # event_loop.close()


print('future result: {!r}'.format(all_done.result()))

scheduling mark done
entering event loop
setting future result to 'the result'
callback 2 invoked at 6702.426535411
callback 1 invoked at 6702.426708303
returning result: 'the result'
closing event loop
future result: 'the result'


The result of the Future is returned by **`await`**.

In [9]:
# asyncio_future_await.py
import asyncio


def mark_done(future, result):
    print('setting future result to {!r}'.format(result))
    future.set_result(result)
    

async def main(loop):
    all_done = asyncio.Future()
    
    print('scheduling mark_done')
    loop.call_soon(mark_done, all_done, 'the result')
    
    result = await all_done
    print('returned result: {!r}'.format(result))
    

event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    pass
    # event_loop.close()

scheduling mark_done
setting future result to 'the result'
returned result: 'the result'


A Future can invoke callbacks when it is completed.

In [10]:
# asyncio_future_callback.py

import asyncio
import functools


def callback(future, n):
    print('{}: future done: {}'.format(n, future.result()))
    
    
async def register_callbacks(all_done):
    print('registering callbacks on future')
    all_done.add_done_callback(functools.partial(callback, n=1))
    all_done.add_done_callback(functools.partial(callback, n=2))
    

async def main(all_done):
    await register_callbacks(all_done)
    print('setting result of future')
    all_done.set_result('the result')
    
    
event_loop = asyncio.get_event_loop()
try:
    all_done = asyncio.Future()
    event_loop.run_until_complete(main(all_done))
finally:
    pass
    #event_loop.close()

registering callbacks on future
setting result of future
1: future done: the result
2: future done: the result


## Executing Tasks Concurrently 
Tasks - subclasses of Future, other coroutines can wait for them and each has a result.

### Starting a Task 
The resulting task will run as part of the concurrent operation

In [11]:
import asyncio


async def task_func():
    print('in task_func')
    return 'the result'


async def main(loop):
    print('creating task')
    task = loop.create_task(task_func())
    print('waiting for {!r}'.format(task))
    return_value = await task
    print('task completed {!r}'.format(task))
    print('return value: {!r}'.format(return_value))
    

event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    pass
    # event_loop.close()

creating task
waiting for <Task pending coro=<task_func() running at <ipython-input-11-e37780e9485b>:4>>
in task_func
task completed <Task finished coro=<task_func() done, defined at <ipython-input-11-e37780e9485b>:4> result='the result'>
return value: 'the result'


### Canceling a Task

In [12]:
# asyncio_cancel_task.py

import asyncio


async def task_func():
    print('in task_func')
    return 'the result'


async def main(loop):
    print('creating task')
    task = loop.create_task(task_func())
    
    print('canceling task')
    task.cancel()
    
    print('canceled task {!r}'.format(task))
    try:
        await task
    except  asyncio.CancelledError:
        print('caught error from canceled task')
    else:
        print('task result: {!r}'.format(task.result()))


event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    pass
    # event_loop.close()

creating task
canceling task
canceled task <Task cancelling coro=<task_func() running at <ipython-input-12-cf37153fb0f9>:6>>
caught error from canceled task


If a task is canceled while it is waiting for another concurrent operation, the task is notified of its cancellation by having a CancelledError exception raised at the point where it is waiting.

In [13]:
# asyncio_cancel_task2.py
import asyncio


async def task_fun():
    print('in task_func, sleeping')
    try:
        await asyncio.sleep(1)
    except asyncio.CancelledError:
        print('task_func was canceled')
        raise
    return 'the result'

    
def task_canceller(t):
    print('in task_canceller')
    t.cancel()
    print('canceled the task')


async def main(loop):
    print('creating task')
    task = loop.create_task(task_func())
    loop.call_soon(task_canceller, task)
    try:
        await task
    except asyncio.CancelledError:
        print('task_func was canceled')
        raise
    return 'the result'
    

event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    pass
    # event_loop.close()

creating task
in task_func
in task_canceller
canceled the task


### Creating Tasks from Coroutines

**`ensure_future()`** returns a Task ties to the execution of a coroutine.

In [14]:
# asyncio_ensure_future.py
import asyncio


async def wrapped():
    print('wrapped')
    return 'result'


async def inner(task):
    print('inner: starting')
    print('inner: waiting for {!r}'.format(task))
    result = await task
    print('inner: task returned {!r}'.format(task))
    
    
async def starter():
    print('starter: creating task')
    task = asyncio.ensure_future(wrapped())
    print('starter: waiting for inner')
    await inner(task)
    print('starter: inner returned')
    

event_loop = asyncio.get_event_loop()
try:
    print('entering event loop')
    result = event_loop.run_until_complete(starter())
finally:
    pass
    # event_loop.close()



entering event loop
starter: creating task
starter: waiting for inner
inner: starting
inner: waiting for <Task pending coro=<wrapped() running at <ipython-input-14-c267531fe386>:5>>
wrapped
inner: task returned <Task finished coro=<wrapped() done, defined at <ipython-input-14-c267531fe386>:5> result='result'>
starter: inner returned


## Composing Coroutines with Control Structures

### Waiting for Multiple Coroutines
dividing one operation into many parts and execute them separately.

In [15]:
# asyncio_wait.py

import asyncio


async def phase(i):
    print('in phase {}'.format(i))
    await asyncio.sleep(0.1 * i)
    print('done with phase {}'.format(i))
    return 'phase {} result'.format(i)


async def main(num_phases):
    print('starting main')
    phases = [
        phase(i)
        for i in range(num_phases)
    ]
    print('waiting for phases to complete')
    completed, pending = await asyncio.wait(phases)
    results = [t.result() for t in completed]
    print('results: {!r}'.format(results))
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(3))
finally:
    pass
    # event_loop.close()

starting main
waiting for phases to complete
in phase 2
in phase 0
in phase 1
done with phase 0
done with phase 1
done with phase 2
results: ['phase 2 result', 'phase 0 result', 'phase 1 result']


In [16]:
# asyncio_wait_timeout.py
import asyncio


async def phase(i):
    print('in phase {}'.format(i))
    try:
        await asyncio.sleep(0.1 * i)
    except asyncio.CancelledError:
        print('phase {} canceled'.format(i))
        raise
    else:
        print('done with phase {}'.format(i))
        return 'phase {} result'.format(i)


async def main(num_phases):
    print('starting main')
    phases = [
        phase(i)
        for i in range(num_phases)
    ]
    print('waiting 0.1 for phases to complete')
    completed, pending = await asyncio.wait(phases, timeout=0.1)
    print('{} completed and {} pending'.format(
        len(completed), len(pending),
    ))
   
    if pending:
        print('canceling tasks')
        for t in pending:
            t.cancel()
    print('exiting main')


event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(3))
finally:
    pass
    # event_loop.close()


starting main
waiting 0.1 for phases to complete
in phase 1
in phase 2
in phase 0
done with phase 0
1 completed and 2 pending
canceling tasks
exiting main
phase 1 canceled
phase 2 canceled


### Gathering Results from Coroutines

In [17]:
# asyncio_gather.py
import asyncio


async def phase1():
    print('in phase1')
    await asyncio.sleep(2)
    print('done with phase1')
    return 'phase1 result'


async def phase2():
    print('in phase2')
    await asyncio.sleep(1)
    print('done with phase2')
    return 'phase2 result'


async def main():
    print('starting main')
    print('waiting for phases to complete')
    results = await asyncio.gather(
        phase1(),
        phase2(),
    )
    print('results: {!r}'.format(results))


event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main())
finally:
    pass
    # event_loop.close()

starting main
waiting for phases to complete
in phase1
in phase2
done with phase2
done with phase1
results: ['phase1 result', 'phase2 result']


### Handling Background Operations as They Finish

In [18]:
# asyncio_as_completed.py
import asyncio


async def phase(i):
    print('in phase {}'.format(i))
    await asyncio.sleep(0.5 - (0.1 * i))
    print('done with phase {}'.format(i))
    return 'phase {} result'.format(i)


async def main(num_phases):
    print('starting main')
    phases = [
        phase(i)
        for i in range(num_phases)
    ]
    print('waiting for phases to complete')
    results = []
    for next_to_complete in asyncio.as_completed(phases):
        answer = await next_to_complete
        print('received answer {!r}'.format(answer))
        results.append(answer)
    print('results: {!r}'.format(results))
    return results


event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(3))
finally:
    pass
    # event_loop.close()

starting main
waiting for phases to complete
in phase 1
in phase 2
in phase 0
done with phase 2
received answer 'phase 2 result'
done with phase 1
received answer 'phase 1 result'
done with phase 0
received answer 'phase 0 result'
results: ['phase 2 result', 'phase 1 result', 'phase 0 result']
